In [43]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, ELU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [2]:
# Загрузка данных
images = np.load('images.npy')
labels = np.load('labels.npy')
images_sub = np.load('images_sub.npy')

In [3]:
labels

array([13, 24,  8, ..., 21,  7, 18])

In [4]:
# Нормализация изображений
images = images / 255.0
images_sub = images_sub / 255.0

# Преобразование меток в категории
labels = tf.keras.utils.to_categorical(labels, num_classes=26)

# Разделение данных на обучающую и проверочную выборки
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.1, random_state=42)

# Создание модели с добавлением регуляризации и изменением архитектуры
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),  # Дополнительный свёрточный слой
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),  # Увеличенный полносвязный слой с регуляризацией
    Dropout(0.5),
    Dense(26, activation='softmax')
])

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Подготовка к расширению данных
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Callback для сохранения лучшей модели
checkpoint = ModelCheckpoint('best_model', monitor='val_accuracy', save_best_only=True, mode='max', save_format='tf')

2024-05-19 05:09:50.911578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31136 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:8c:00.0, compute capability: 7.0


In [5]:
# Обучение с использованием расширения данных, ранней остановки и сохранения лучшей модели
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          validation_data=(X_val, y_val),
          steps_per_epoch=len(X_train) // 32,
          epochs=100,
          callbacks=[early_stopping, checkpoint])


2024-05-19 05:10:05.950762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/100


2024-05-19 05:10:07.624079: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2024-05-19 05:10:09.293198: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f01d8971a90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-19 05:10:09.293267: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2024-05-19 05:10:09.472540: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-19 05:10:10.568626: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


562/562 [==============================] - ETA: 0s - loss: 4.8423 - accuracy: 0.0474

2024-05-19 05:10:31.435784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 05:10:31.898209: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


...

562/562 [==============================] - 19s 34ms/step - loss: 2.0959 - accuracy: 0.3891 - val_loss: 1.3778 - val_accuracy: 0.6310
Epoch 95/100
562/562 [==============================] - 18s 31ms/step - loss: 2.1023 - accuracy: 0.3833 - val_loss: 1.4163 - val_accuracy: 0.6140
Epoch 96/100
561/562 [============================>.] - ETA: 0s - loss: 2.0815 - accuracy: 0.3885

2024-05-19 05:39:38.033260: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 05:39:38.716345: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


562/562 [==============================] - 20s 35ms/step - loss: 2.0821 - accuracy: 0.3883 - val_loss: 1.3814 - val_accuracy: 0.6370
Epoch 97/100
562/562 [==============================] - 18s 31ms/step - loss: 2.0860 - accuracy: 0.3895 - val_loss: 1.4302 - val_accuracy: 0.6070
Epoch 98/100
562/562 [==============================] - 18s 32ms/step - loss: 2.0718 - accuracy: 0.3872 - val_loss: 1.3598 - val_accuracy: 0.6350
Epoch 99/100
562/562 [==============================] - 18s 31ms/step - loss: 2.0838 - accuracy: 0.3914 - val_loss: 1.3638 - val_accuracy: 0.6275
Epoch 100/100
562/562 [==============================] - 18s 31ms/step - loss: 2.0814 - accuracy: 0.3928 - val_loss: 1.3542 - val_accuracy: 0.6260


In [17]:
# Обучение с использованием расширения данных, ранней остановки и сохранения лучшей модели
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          validation_data=(X_val, y_val),
          steps_per_epoch=len(X_train) // 32,
          epochs=100,
          callbacks=[early_stopping, checkpoint])

2024-05-19 05:48:59.156436: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/100
561/562 [============================>.] - ETA: 0s - loss: 2.0891 - accuracy: 0.3855

2024-05-19 05:49:19.855427: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 05:49:20.277944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


...

562/562 [==============================] - 19s 34ms/step - loss: 1.8189 - accuracy: 0.4734 - val_loss: 1.0340 - val_accuracy: 0.7415
Epoch 75/100
562/562 [==============================] - 18s 32ms/step - loss: 1.8081 - accuracy: 0.4709 - val_loss: 1.0614 - val_accuracy: 0.7360
Epoch 76/100
562/562 [==============================] - 18s 32ms/step - loss: 1.8339 - accuracy: 0.4698 - val_loss: 1.0663 - val_accuracy: 0.7370
Epoch 77/100
562/562 [==============================] - ETA: 0s - loss: 1.8079 - accuracy: 0.4681

2024-05-19 06:12:09.083648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 06:12:09.465704: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


562/562 [==============================] - 20s 35ms/step - loss: 1.8079 - accuracy: 0.4681 - val_loss: 1.0212 - val_accuracy: 0.7515
Epoch 78/100
562/562 [==============================] - 18s 32ms/step - loss: 1.8102 - accuracy: 0.4693 - val_loss: 1.0638 - val_accuracy: 0.7380
Epoch 79/100
562/562 [==============================] - 18s 31ms/step - loss: 1.8026 - accuracy: 0.4811 - val_loss: 1.0458 - val_accuracy: 0.7315
Epoch 80/100
562/562 [==============================] - 18s 32ms/step - loss: 1.7970 - accuracy: 0.4764 - val_loss: 1.0760 - val_accuracy: 0.7225
Epoch 81/100
562/562 [==============================] - 18s 31ms/step - loss: 1.8116 - accuracy: 0.4714 - val_loss: 1.0588 - val_accuracy: 0.7200
Epoch 82/100
562/562 [==============================] - 18s 31ms/step - loss: 1.7988 - accuracy: 0.4726 - val_loss: 1.0685 - val_accuracy: 0.7250
Epoch 83/100
562/562 [==============================] - 18s 31ms/step - loss: 1.8028 - accuracy: 0.4745 - val_loss: 1.0640 - val_accuracy

In [19]:
# Обучение с использованием расширения данных, ранней остановки и сохранения лучшей модели
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(datagen.flow(X_train, y_train, batch_size=64),
          validation_data=(X_val, y_val),
          steps_per_epoch=len(X_train) // 64,
          epochs=10,
          callbacks=[early_stopping, checkpoint])

2024-05-19 06:19:28.020189: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/10
281/281 [==============================] - 17s 60ms/step - loss: 1.7871 - accuracy: 0.4813 - val_loss: 1.0186 - val_accuracy: 0.7350
Epoch 2/10
281/281 [==============================] - 17s 60ms/step - loss: 1.7728 - accuracy: 0.4844 - val_loss: 1.0336 - val_accuracy: 0.7365
Epoch 3/10
281/281 [==============================] - 17s 60ms/step - loss: 1.7588 - accuracy: 0.4813 - val_loss: 1.0264 - val_accuracy: 0.7405
Epoch 4/10
281/281 [==============================] - 17s 61ms/step - loss: 1.7665 - accuracy: 0.4865 - val_loss: 1.0057 - val_accuracy: 0.7495
Epoch 5/10
281/281 [==============================] - 17s 60ms/step - loss: 1.7602 - accuracy: 0.4839 - val_loss: 0.9964 - val_accuracy: 0.7455
Epoch 6/10
281/281 [==============================] - 17s 60ms/step - loss: 1.7596 - accuracy: 0.4891 - val_loss: 1.0135 - val_accuracy: 0.7440
Epoch 7/10
281/281 [==============================] - 17s 60ms/step - loss: 1.7572 - accuracy: 0.4901 - val_loss: 1.0039 - val_accuracy:

In [20]:
# Обучение с использованием расширения данных, ранней остановки и сохранения лучшей модели
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(datagen.flow(X_train, y_train, batch_size=64),
          validation_data=(X_val, y_val),
          steps_per_epoch=len(X_train) // 64,
          epochs=100,
          callbacks=[early_stopping, checkpoint])

2024-05-19 06:23:19.738592: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/100
281/281 [==============================] - 17s 61ms/step - loss: 1.7636 - accuracy: 0.4861 - val_loss: 0.9955 - val_accuracy: 0.7470
Epoch 2/100
281/281 [==============================] - 17s 60ms/step - loss: 1.7489 - accuracy: 0.4885 - val_loss: 1.0134 - val_accuracy: 0.7405
Epoch 3/100
281/281 [==============================] - 17s 60ms/step - loss: 1.7488 - accuracy: 0.4893 - val_loss: 0.9997 - val_accuracy: 0.7460
Epoch 4/100
281/281 [==============================] - 17s 60ms/step - loss: 1.7468 - accuracy: 0.4920 - val_loss: 0.9971 - val_accuracy: 0.7450
Epoch 5/100
281/281 [==============================] - 17s 60ms/step - loss: 1.7357 - accuracy: 0.4938 - val_loss: 1.0054 - val_accuracy: 0.7365
Epoch 6/100
281/281 [==============================] - 17s 60ms/step - loss: 1.7258 - accuracy: 0.4972 - val_loss: 1.0047 - val_accuracy: 0.7455
Epoch 7/100
281/281 [==============================] - 17s 60ms/step - loss: 1.7485 - accuracy: 0.4845 - val_loss: 0.9869 - val_ac

2024-05-19 06:26:59.858051: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 06:27:00.243992: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


...

281/281 [==============================] - 19s 66ms/step - loss: 1.7103 - accuracy: 0.4984 - val_loss: 0.9588 - val_accuracy: 0.7625
Epoch 25/100
281/281 [==============================] - 17s 60ms/step - loss: 1.7109 - accuracy: 0.5047 - val_loss: 1.0033 - val_accuracy: 0.7385
Epoch 26/100
281/281 [==============================] - ETA: 0s - loss: 1.7115 - accuracy: 0.5023

2024-05-19 06:30:45.340438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 06:30:45.725800: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


281/281 [==============================] - 18s 66ms/step - loss: 1.7115 - accuracy: 0.5023 - val_loss: 0.9418 - val_accuracy: 0.7675
Epoch 27/100
281/281 [==============================] - 17s 61ms/step - loss: 1.7129 - accuracy: 0.5032 - val_loss: 0.9615 - val_accuracy: 0.7540
Epoch 28/100
281/281 [==============================] - 17s 61ms/step - loss: 1.7119 - accuracy: 0.4970 - val_loss: 0.9583 - val_accuracy: 0.7565
Epoch 29/100
281/281 [==============================] - 17s 60ms/step - loss: 1.6972 - accuracy: 0.5036 - val_loss: 0.9592 - val_accuracy: 0.7510
Epoch 30/100
281/281 [==============================] - 17s 60ms/step - loss: 1.6836 - accuracy: 0.5112 - val_loss: 0.9956 - val_accuracy: 0.7350
Epoch 31/100
281/281 [==============================] - 17s 60ms/step - loss: 1.6960 - accuracy: 0.5059 - val_loss: 0.9543 - val_accuracy: 0.7610
Epoch 32/100
281/281 [==============================] - 17s 60ms/step - loss: 1.7024 - accuracy: 0.5052 - val_loss: 0.9654 - val_accuracy

In [22]:
# Обучение с использованием расширения данных, ранней остановки и сохранения лучшей модели
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(datagen.flow(X_train, y_train, batch_size=128),
          validation_data=(X_val, y_val),
          steps_per_epoch=len(X_train) // 128,
          epochs=100,
          callbacks=[early_stopping, checkpoint])

2024-05-19 06:35:14.457823: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/100
140/140 [==============================] - 17s 117ms/step - loss: 1.6900 - accuracy: 0.5080 - val_loss: 0.9482 - val_accuracy: 0.7580
Epoch 2/100
140/140 [==============================] - 16s 117ms/step - loss: 1.6719 - accuracy: 0.5122 - val_loss: 0.9364 - val_accuracy: 0.7625
Epoch 3/100
140/140 [==============================] - ETA: 0s - loss: 1.6778 - accuracy: 0.5096

2024-05-19 06:36:04.463848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 06:36:04.856494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


...

140/140 [==============================] - 18s 127ms/step - loss: 1.6396 - accuracy: 0.5207 - val_loss: 0.9071 - val_accuracy: 0.7765
Epoch 30/100
140/140 [==============================] - 16s 117ms/step - loss: 1.6218 - accuracy: 0.5233 - val_loss: 0.8982 - val_accuracy: 0.7680
Epoch 31/100
140/140 [==============================] - 16s 116ms/step - loss: 1.6411 - accuracy: 0.5204 - val_loss: 0.9083 - val_accuracy: 0.7645
Epoch 32/100
140/140 [==============================] - 16s 117ms/step - loss: 1.6282 - accuracy: 0.5210 - val_loss: 0.9204 - val_accuracy: 0.7560
Epoch 33/100
140/140 [==============================] - 16s 116ms/step - loss: 1.6379 - accuracy: 0.5172 - val_loss: 0.9038 - val_accuracy: 0.7640
Epoch 34/100
140/140 [==============================] - 16s 116ms/step - loss: 1.6407 - accuracy: 0.5197 - val_loss: 0.9112 - val_accuracy: 0.7645
Epoch 35/100
140/140 [==============================] - 16s 116ms/step - loss: 1.6206 - accuracy: 0.5196 - val_loss: 0.8957 - val_a

2024-05-19 06:45:27.369235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 06:45:27.757548: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


140/140 [==============================] - 18s 130ms/step - loss: 1.6357 - accuracy: 0.5213 - val_loss: 0.9079 - val_accuracy: 0.7800


In [40]:
# Обучение с использованием расширения данных, ранней остановки и сохранения лучшей модели
early_stopping = EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)
model.fit(datagen.flow(X_train, y_train, batch_size=128),
          validation_data=(X_val, y_val),
          steps_per_epoch=len(X_train) // 128,
          epochs=400,
          callbacks=[early_stopping, checkpoint])

2024-05-19 08:39:16.534667: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/400
140/140 [==============================] - 16s 117ms/step - loss: 1.4464 - accuracy: 0.5770 - val_loss: 0.7852 - val_accuracy: 0.8025
Epoch 2/400
140/140 [==============================] - 16s 116ms/step - loss: 1.4659 - accuracy: 0.5680 - val_loss: 0.7661 - val_accuracy: 0.8130
Epoch 3/400
140/140 [==============================] - 16s 116ms/step - loss: 1.4797 - accuracy: 0.5672 - val_loss: 0.8004 - val_accuracy: 0.7935
Epoch 4/400
140/140 [==============================] - ETA: 0s - loss: 1.4607 - accuracy: 0.5709

2024-05-19 08:40:22.046387: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 08:40:22.430892: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


...

2024-05-19 09:07:34.634207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 09:07:35.043632: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


140/140 [==============================] - 18s 127ms/step - loss: 1.4018 - accuracy: 0.5903 - val_loss: 0.7305 - val_accuracy: 0.8210
Epoch 105/400
140/140 [==============================] - 16s 115ms/step - loss: 1.4055 - accuracy: 0.5858 - val_loss: 0.7424 - val_accuracy: 0.8085
Epoch 106/400
140/140 [==============================] - 16s 116ms/step - loss: 1.4252 - accuracy: 0.5814 - val_loss: 0.7322 - val_accuracy: 0.8170
Epoch 107/400
140/140 [==============================] - 16s 115ms/step - loss: 1.4074 - accuracy: 0.5834 - val_loss: 0.7435 - val_accuracy: 0.8045
Epoch 108/400
140/140 [==============================] - 16s 116ms/step - loss: 1.4141 - accuracy: 0.5854 - val_loss: 0.7409 - val_accuracy: 0.8100
Epoch 109/400
140/140 [==============================] - 16s 116ms/step - loss: 1.4164 - accuracy: 0.5833 - val_loss: 0.7368 - val_accuracy: 0.8075
Epoch 110/400
 16/140 [==>...........................] - ETA: 14s - loss: 1.4170 - accuracy: 0.5935

KeyboardInterrupt: 

In [ ]:
# Загрузка текущей модели (предполагается, что модель уже сохранена в файл 'best_model')
model = load_model('best_model')

# Компиляция модели с новыми параметрами
model.compile(optimizer=Adam(learning_rate=0.000001),  # более низкая скорость обучения для тонкой настройки
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Создание генератора данных для аугментации изображений
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Повторное использование коллбэков для тонкой настройки
checkpoint = ModelCheckpoint('best_model_finetuned', monitor='val_accuracy', save_best_only=True, mode='max', save_format='tf')
early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Обучение модели с использованием расширения данных
model.fit(datagen.flow(X_train, y_train, batch_size=64),  # уменьшен размер батча для более точной настройки
          validation_data=(X_val, y_val),
          steps_per_epoch=len(X_train) // 64,  # обновлено количество шагов на эпоху
          epochs=200,  # уменьшено количество эпох для тонкой настройки
          callbacks=[early_stopping, checkpoint])


2024-05-19 10:12:31.963640: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/200
281/281 [==============================] - ETA: 0s - loss: 1.4109 - accuracy: 0.5827

2024-05-19 10:12:51.715309: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 10:12:52.143986: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model_finetuned/assets


INFO:tensorflow:Assets written to: best_model_finetuned/assets


...

281/281 [==============================] - 19s 66ms/step - loss: 1.3619 - accuracy: 0.5990 - val_loss: 0.7124 - val_accuracy: 0.8225
Epoch 69/200
281/281 [==============================] - 17s 61ms/step - loss: 1.3746 - accuracy: 0.5966 - val_loss: 0.7130 - val_accuracy: 0.8210
Epoch 70/200
281/281 [==============================] - 17s 62ms/step - loss: 1.3726 - accuracy: 0.5970 - val_loss: 0.7124 - val_accuracy: 0.8220
Epoch 71/200
281/281 [==============================] - 17s 61ms/step - loss: 1.3643 - accuracy: 0.6014 - val_loss: 0.7122 - val_accuracy: 0.8215
Epoch 72/200
281/281 [==============================] - 17s 61ms/step - loss: 1.3687 - accuracy: 0.6021 - val_loss: 0.7117 - val_accuracy: 0.8205
Epoch 73/200
281/281 [==============================] - 17s 60ms/step - loss: 1.3535 - accuracy: 0.6023 - val_loss: 0.7120 - val_accuracy: 0.8190
Epoch 74/200
281/281 [==============================] - 17s 61ms/step - loss: 1.3643 - accuracy: 0.5962 - val_loss: 0.7129 - val_accuracy

2024-05-19 10:58:06.190526: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 10:58:06.578095: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model_finetuned/assets


INFO:tensorflow:Assets written to: best_model_finetuned/assets


281/281 [==============================] - 18s 65ms/step - loss: 1.3631 - accuracy: 0.5993 - val_loss: 0.7050 - val_accuracy: 0.8230
Epoch 158/200
281/281 [==============================] - 18s 62ms/step - loss: 1.3583 - accuracy: 0.6031 - val_loss: 0.7047 - val_accuracy: 0.8190
Epoch 159/200
281/281 [==============================] - 17s 61ms/step - loss: 1.3624 - accuracy: 0.5991 - val_loss: 0.7048 - val_accuracy: 0.8205
Epoch 160/200
281/281 [==============================] - 17s 60ms/step - loss: 1.3382 - accuracy: 0.6010 - val_loss: 0.7044 - val_accuracy: 0.8175
Epoch 161/200
281/281 [==============================] - 17s 61ms/step - loss: 1.3529 - accuracy: 0.6041 - val_loss: 0.7042 - val_accuracy: 0.8190
Epoch 162/200
113/281 [===========>..................] - ETA: 10s - loss: 1.3782 - accuracy: 0.5962

In [46]:
# Загрузка текущей модели
model = load_model('best_model')

# Планировщик скорости обучения, который уменьшит скорость обучения, если остановка в улучшении
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.000001, verbose=1)

# Компиляция модели с новыми параметрами
model.compile(optimizer=Adam(learning_rate=0.0001),  # Поднял скорость для видимости изменений
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Создание генератора данных для аугментации изображений
datagen = ImageDataGenerator(
    rotation_range=30,  # Увеличено для большей аугментации
    width_shift_range=0.3,  # Увеличено для большей аугментации
    height_shift_range=0.3,  # Увеличено для большей аугментации
    shear_range=0.3,  # Увеличено для большей аугментации
    zoom_range=0.3,  # Увеличено для большей аугментации
    horizontal_flip=True,
    fill_mode='nearest'
)

# Обновление коллбэков
checkpoint = ModelCheckpoint('best_model_finetuned', monitor='val_accuracy', save_best_only=True, mode='max', save_format='tf')
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)  # Уменьшено терпение для более ранней остановки

# Обучение модели с использованием расширения данных
model.fit(datagen.flow(X_train, y_train, batch_size=64),
          validation_data=(X_val, y_val),
          steps_per_epoch=len(X_train) // 64,
          epochs=100,  # Уменьшено количество эпох
          callbacks=[checkpoint, early_stopping, reduce_lr])


2024-05-19 09:53:55.987068: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/100
281/281 [==============================] - ETA: 0s - loss: 2.0486 - accuracy: 0.4041

2024-05-19 09:54:16.266154: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]
2024-05-19 09:54:16.693722: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,256]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: best_model_finetuned/assets


INFO:tensorflow:Assets written to: best_model_finetuned/assets


281/281 [==============================] - 21s 68ms/step - loss: 2.0486 - accuracy: 0.4041 - val_loss: 0.8576 - val_accuracy: 0.7870 - lr: 1.0000e-04
Epoch 2/100
281/281 [==============================] - 17s 62ms/step - loss: 2.0165 - accuracy: 0.4095 - val_loss: 0.8829 - val_accuracy: 0.7760 - lr: 1.0000e-04
Epoch 3/100
281/281 [==============================] - 17s 62ms/step - loss: 2.0227 - accuracy: 0.4119 - val_loss: 0.8779 - val_accuracy: 0.7800 - lr: 1.0000e-04
Epoch 4/100
281/281 [==============================] - 17s 62ms/step - loss: 2.0261 - accuracy: 0.4102 - val_loss: 0.9045 - val_accuracy: 0.7730 - lr: 1.0000e-04
Epoch 5/100
281/281 [==============================] - 17s 62ms/step - loss: 2.0287 - accuracy: 0.4063 - val_loss: 0.9045 - val_accuracy: 0.7685 - lr: 1.0000e-04
Epoch 6/100
281/281 [==============================] - 17s 62ms/step - loss: 2.0223 - accuracy: 0.4079 - val_loss: 0.9275 - val_accuracy: 0.7570 - lr: 1.0000e-04
Epoch 7/100
281/281 [===================

In [49]:
# Загрузка лучшей модели
model = load_model('best_model')

predictions = model.predict(images_sub)
predicted_classes = np.argmax(predictions, axis=1)

# Создание DataFrame с предсказаниями
submission_df = pd.DataFrame({
    'Id': np.arange(len(predicted_classes)),  # Создаем столбец Id с последовательными номерами от 0 до N-1
    'Category': predicted_classes             # Столбец Category с предсказанными классами
})

# Сохранение DataFrame в CSV файл
submission_df.to_csv('submission_13.csv', index=False)


1563/1563 [==============================] - 4s 2ms/step
